# Locality Sensitive Hashing

Lindsey Oberhelman and Niklas Tillenburg

## The Assignment

Implement the LSH algorithm and apply it to the data.
•Tune it (signature length, number of bands, number of rows per band)•Randomize, optimize, benchmark, polish the code, •Deliver your code, including lines which:
1. load the file user_movie.npy(don’t include this file in your submission!)
2. dump results to a text file ans.txt(just a csv list of records: user1, user2)
3. set the random seed to a specific value, eg.: np.random.seed(seed=17)

In [1]:
import numpy as np
import sys
from scipy import sparse
import itertools
import resource
import time as time
  

In [2]:
def sparse_matrix():
    """
    Loads the data and make a sparse matrix using the scipy module. 
    Every user movie pair in the file is represented as a '1'.
    """
    # The format of the file is Col1: Users  Col2: Movies
    data = np.load('user_movie.npy')

    # Get the total amount of users and total amount of movies rated
    users = data[:, 0]
    movies = data[:, 1]

    # find the largest user and movie ID and since python starts at 0 need add 1 
    number_users = np.max(users)+1
    number_movies = np.max(movies)+1
   
    # if a user rated a movie put into the sparse matrix a '1'
    mat_values = np.ones(data.shape[0])

    # This saves memory and makes Jaccard calculation easier
    sparse_matrix = sparse.csc_matrix((mat_values, (movies, users)), shape=(number_movies, number_users), dtype='b')

    return sparse_matrix, number_users, number_movies

## Minhashing

Minhashing takes a random permutation of the column of movies and hashes the index of the first row for each user in which the column has a value 1. This is repeated for n permutations to construct a minihash signature for each user. These are then combined for all users to construct a signature matrix that will have the same number of columns as there are users and as many rows as permutations n were selected.
As the data sets become extremely large, minihasing can become computationally expensive for large datasets as each permutation requires the dataset to be sorted. The big O notation O(n$^{2}$) where n is the number of things you are comparing. This can be infeasible or even impossible to achieve on extremely large datasets. However since we will combine this with LSH algorithm it will be a bit faster.
Our code:
<br>

Take a random permuatation of the column of movies (new_row_permutation)
<br>
Then the code locates the first position where the user rated a movie records that movie number so each column is a user and each row of that column is a different permuation of movies.
<br>
Our code repeats that for 100 permuations becuase when we tested for 80, 100, 120. A signature matrix was the best we could do in terms of the time. As we mentioned minihashing can become expensive. In this set up it makes the signature in under 3 minutes. 

## Signatures

In [3]:
def make_signatures(S, number_users, number_movies, random_seed):
    """
    Create the signature matrix using minhashing. By usage of random permutations of the rows of the sparse
    data matrix.
    num_permutations: 100 from the text book
    Inputs: The sparse matrix, the number of users, the number of movies, a random seed
    """
    num_permutations = 100

    # allocate memory, force to int16
    sign_matrix = np.zeros((num_permutations, number_users)).astype('int16')
    
    # now get the 100 permutations for the signatures
    for perm in range(num_permutations):
        
        # set the random seed such that our results are the same but also enables different permutations
        r_seed = int(perm * random_seed)
        np.random.seed(r_seed)

        # create a new random permutation of the movies
        new_row_permutation = np.random.permutation(number_movies)

        # Swap the sparse matrix rows with the new row permuation
        perm_sparse = S[new_row_permutation, :]

        # Get the position of the first '1' in each column (user) and save the position of it in the signature matrix
        for j in range(number_users):
            
            #Hashing
            position = perm_sparse.indices[perm_sparse.indptr[j]:perm_sparse.indptr[j + 1]].min()
            sign_matrix[perm, j] = position

    return sign_matrix

## Jaccard Similarity

The Jaccard similarity calculates the similarity of two users by taking their intersection, the number of movies they have both watched, divided by their union, the total number of movies they have watched combined. The larger the Jaccard similarity the more probable it is that the two users are similar. We use the Jaccard Similarity to determine unique and candidate pairs and to determine true pairs in the final check as well. For the signature matrix we set the Jaccard similarity limit to be 0.485 because it allows more false negatives to pass through to the final check. When we tested our code with 0.5 for the signature similarity check and the sparse matrix check we found that there were many more false negatives and our code only found 430 pairs and we still had time left so we tuned algorithm to allow more potentially similar candidate pairs through to the final check.

In [4]:
def j_sim_signature(user1, user2, sign_matrix):
    """    
    This function calculates the similarity between two users (user1 and user2) using the signature matrix   
    Inputs: The two users that need to compare, and the signature matrix
    """
    union = len(sign_matrix[:, user1])
    intersection = float(np.count_nonzero(sign_matrix[:, user1] == sign_matrix[:, user2]))
    similar = intersection/union
    return similar

## Locality Sensitive Hashing (LSH)

Now that Minhashing has reduced the size of the data, LSH generates hash codes that have the property of being similar for near by users. One general approach to LSH is to “hash” items so that similar items are more likely to be hashed to the same "bucket" than dissimilar items. Then it takes any pair that hashed to the same bucket and checks only those pairs for true similarity. This algorithm works on the hope that that most of the dissimilar pairs will not be hased to the same bucket. False positives are those dissimilar pairs that do hash to the same bucket. The algorithm also aims to limit the number of false negatives or pairs that are similar and do not get hashed to the same bucket.  The algortihm first divides the signature matrix into bands consisting of r rows each. Each band,has a hash function that takes vectors of r integers(the portion of one column within that band) and hashes them to some large number of buckets. The algorithm uses the same hash function for all the bands, but a separate bucket array for each band, so columns with the same vector in different bands will not hash to the same bucket.
<br>
For our code we:
<br>
First we create a list of candidate pairs and we whose similarity must be measured. 
<br>
All pairs that are not candidates are assumed to be not similar
<br>
We perform LSH on our signature matrix by creating a large number of hash fucntions
<br>
for each hash function we hash columns to buckets in which everything in the bucket is a candidate pair
A pair becomes a candidate pair when the hash function puts one or more of the signatures in the bucket
Want each bucket to have relatively few candidate pairs in them
Can't use too many buckets becuase then pairs that are not truly similar will not end up in the same bucket


We determined the number of (b) bands and (r) rows by experiementing with differnet configurations 100/20/5 (like in the book), 100/10/10, 100/25/4. 
We looked at the probabilities:

Recall that the probability of two documents having the same MinHash value on any row is exactly J(A,B) and every row is independent of the other.
<br>
The probability of the two users haveing the same MinHash values on any of the r rows of a given band is therefore $J(A,B)^{r}$  
<br>
The probability that two documents have atleast one different Minhash on any given row is 1-J(A,B)$^{r}$
<br>
Then the probability of the teo users having atleast one different Minhash value on any of the bands  is (1-J(A,B)$^{r})^{b}$
<br>
From this we can determine that the probability that we need to hash the two users is 1-(1-J(A,B)$^{r})^{b}$
<br>
<br>
We used the probability that we need to hash the two users to determine the best number of rows and bands by making probability distributions of different configurations. We determined that 100/25/4 was the best configuration to allow for the maximum correct pairs and to minimize the number of false negatives and false positives. However our code is not efficient enough to run this configuration in that time. When we ran it on our computer we found 790 pairs in 30 minutes. We instead choose to use the 100/20/5 configuration as it finshed running under the mark.

In [5]:
def lsh(sign_matrix, total_permutation = 100):
    """
    Need a function to perform the lsh algorithm like in 
    3.4.1 in book signature length of 100, with 20 bands 5 rows like in the book.
    http://localhost:8888/notebooks/Desktop/AiDM/lsh.ipynb#d 
    Inputs: Signature matrix, total number of permutations 
    """
    # buckets for sigs to be hashed into LSH with banding
    number_bands = 20
    number_rows = 5

    # each bucket as a list, it is a list because of the different size bucket tuples
    buckets = []

    current_row = 0
    
    for b in range(number_bands):
        
        # Make the band 
        band = sign_matrix[current_row:int(number_rows) + current_row, :]
        current_row += int(number_rows)

        # obtain a array in which duplicate vectors in the band are put into the same bucket; the candidate pairs
        indices = np.ravel_multi_index(band.astype(int), band.max(1).astype(int) + 1)
        sidx = indices.argsort()
        sorted_indices = indices[sidx]
        buckets_array = np.array(np.split(sidx, np.nonzero(sorted_indices[1:] > sorted_indices[:-1])[0] + 1))

        # Only record the buckets with more than 1 user in it
        for position in range(len(buckets_array)):
            if len(buckets_array[position]) > 1:
                
                buckets.append(buckets_array[position])

    x = map(tuple, buckets)
    
    buckets = set(x)
 
    buckets = list(buckets)
    return buckets

   

In [6]:
def find_unique_sets(bucket_list, sign_matrix):
    """
    This function find the unique candidate pairs from the hashed buckets and determines 
    if they are similar using the jaccard similarity. If it is larger then 0.5 for the signature 
    matrix then it is counted. Removes the potentail doubles.
    Inputs: list of buckets and the signature matrix
    """

    unique_set = set()
    
    for b in range(len(bucket_list)):
        
        # Generate all the combinations of the pairs per bucket
        large_user_pair = set(pair for pair in itertools.combinations(bucket_list[b], 2))
        
        # See if they are already in the set
        large_user_pair = large_user_pair.difference(unique_set)
        
        for user_pair in large_user_pair:
            
            similarity = j_sim_signature(user_pair[0], user_pair[1], sign_matrix)
            
            if similarity >= 0.4:
                unique_set.add(user_pair)
                
    return unique_set


In [7]:
def jaccards_similarity(user1, user2, sparse_matrix):
    """     
    Calculate and return the Jaccard similarity between two users (user1 and user2) with the sparse matrix   
    Inputs: Two unique users and the sparese matrix
    """
    intersection = np.sum(sparse_matrix[:, user1] & sparse_matrix[:, user2])
    union = np.sum(sparse_matrix[:, user1] | sparse_matrix[:, user2])
    jacard_sim = float(intersection) / float(union)
    return jacard_sim

Once the unique pairs are determined by calculating the jaccard similarity on the signature matrix then we check the two users using the original sparse matrix. We first make a copy of the original matrix so that we do not edit it. Then we run thorugh the unique sets (unique_sets) and check the jaccard similarity and check for doubles. This ensures that we are getting correct pairs. 

In [8]:
def check_true_similarity(unique_sets, sparse):
    """
    This function checks the jaccard similarity of each unique pair using the entire movie data
    for the users in the sparse Matrix
    Inputs: The set of unique pairs, the sparse matrix
    """

    copy_unique_sets = unique_sets 
    sorted_unique_sets = sorted(copy_unique_sets)
    
    #Need to make the sparse matrix into an array in order for the | and & to work
    original_matrix = sparse.toarray()
    pair_list = []
    
    # check if the similarity is really > 0.5 with the jaccard similarity 
    for pairs in unique_sets:
        
        #This ensures that false positives and doubles are taken care of.
        # if user1 < user2 and j add to the list of definite pairs.
        if pairs[0] < pairs[1]:
            
            sim = jaccards_similarity(pairs[0], pairs[1], original_matrix)
            
            # and jaccard similarity > 0.5
            if sim > 0.5:
                
                pair_list.append((pairs[0], pairs[1]))

        # if user 2 is larger than user 1  
        elif pairs[0] > pairs[1]:
            
            # check to see if pair is in the set 
            if (pairs[1], pairs[0]) in copy_unique_sets:
                
                continue
                
            # if not then calcualte the jaccard and save it
            else:
                
                sim = jaccards_similarity(pairs[0], pairs[1], original_matrix)
                
                if sim > 0.5:
                
                    pair_list.append((pairs[1], pairs[0]))

    #sort user pair list so that it is easy to compare to all_the_pairs.txt
    pair_list = sorted(pair_list)
    
    # write to txt file
    with open('results.txt', 'w') as f:
        f.write('\n'.join('%s,%s' % user_pair for user_pair in pair_list))
    f.close()
    print('User Pair: ', len(pair_list))

In [9]:
def main():
    """
    This funciton: Starts the timer, Loads the data, Creates the sparse matrix,
    Makes the signatures and hashing them into buckets, Then find similar signatures and
    checks the jaccard similarity with the sparse matrix, Finally creates the file of pairs
    """
    #Start the timer
    start_time = time.time()
    
    #Import the data and make the sparse matrix
    S,u,m = sparse_matrix()
   
    #set a random seed for the permuation of movies and make the signature matrix
    signature_matrix = make_signatures(S, u, m, 1234) 
    print('done with signature')
    
    
    #hash the signatures to buckets
    buckets = lsh(signature_matrix)
    #print('The number of buckets is {}'.format(len(buckets)))
    
    #Find the unique pairs using j similarity on the signature matrix
    unique_set = find_unique_sets(buckets, signature_matrix)
    #print('The number of unique pairs is {}'.format(len(unique_set)))
    
    #Checks the user pairs similarity on the sparse matrix
    print('checking the true similarity of the pairs')
    check_true_similarity(unique_set, S)
    total_time = (time.time()-start_time)/60
    print("The total time in minutes to run is:{}".format(total_time))

main()    

done with signature
checking the true similarity of the pairs
User Pair:  655
The total time in minutes to run is:5.942305966218313


# Results
On out laptops our code produces 

430 pairs in about 5 minutes. 
566 pairs in about 6 minutes.
655 pairs in about 9 minutes.
700+ in over 15 minutes.

All of them were correct. We checked our results against the all_similar_pairs.txt in the OLD assignment folder and we found that most of our pairs are on that list. So we know that our signature we are atleast generating correct pairs. We suspect that if we perform more hashings we will get more pairs however our code is not efficient enough to accomplish this in 15 minutes. The ideal ratio of bands to rows is 100 permutations , 25 bands, and 4 rows. We discovered this doing the research into the probability of pairs and making the plots. Below you can see the comparison between our results and the file of all the similar pairs from last year and all of our pairs are in the results from last year so we are generating correct pairs. 
<br>
## Computaional Expenses
Time: Our code ran in about five minutes on a average laptop. The time compexity for this code was a bit hard to determine. For the loading of the data loading the data seemed to scale with the size of the file. Creating the signatire matrix scaled with O(n$*u$) where n is the number of permuations you want and u is the number of users. And since the code loops through the the users for every number of permuations. Then for the lsh function it scaled with the number of bands and rows we decided to go with O(r$*b).$ For the final check O($s*sparse$) It was the number of sets and then the time it took to access all the data in the sparse matrix. 
<br>
Memory: We conserve memory in our algorithm by using a sparse matrix, which deletes all the zeros and only saves the movies each user has seen as ones. This cuts out a lot of memory that in such a huge data set is crucial. The memory 

In [10]:
results =  np.loadtxt('results.txt', delimiter = ',')
pairs = np.loadtxt('all_similar_pairs.txt', delimiter = ',')
pairs = pairs[:,:2]
any(np.setdiff1d(results, pairs))

False

## Improvements
Ways to imporve the code would be to add a transitive property where if user 1 and user 2 are both similar to user 3 then they are similar. This would improve the speed and limit the number of checks. If we are also able to make the code faster we could generate more unique pairs from the signature by lowering the jaccard similarity limit and being able to complete it in time.